In [1]:
import cffi
import os
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio import Entrez
import time
import argparse
import sys
from src.SequenceProcessor import SequenceProcessor
from src.FileHandler import FileHandler
from src.pytrsomix import SeqAnalyzer,TRScalculator
from src.stats import Stats
from src.BlastProcessor import BLASTProcessor
import pathlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import re

In [2]:
from collections import Counter

# Wczytaj plik
file_path = "/home/hubert/TRS-omix_new/Results_passing/Mycobacterophagesfull_results_L100_R100_c0.8/blast_output/modified_blast/taxids_not_in_clusters_combined_sequences_unique_blastn_out.txt"

with open(file_path, "r") as file:
    lines = file.readlines()

# Pobierz identyfikatory z ostatniej kolumny
last_column = [line.strip().split()[-1] for line in lines]

# Policz wystąpienia
counted = Counter(last_column)

# Wyświetl wyniki
for identifier, count in counted.items():
    print(f"{identifier} : {count}")


2656574 : 2
2499055 : 2
2499068 : 2
2499062 : 2
2250407 : 3
2250403 : 2
2250422 : 3
2250359 : 2
2041540 : 3
1983574 : 2
1897426 : 2
1897767 : 1
3077148 : 2
3035390 : 2
1873890 : 3
1567469 : 2
1541820 : 3
1458715 : 3
2871533 : 3
2863848 : 3
2768135 : 2
2768134 : 3
1034119 : 2
1034126 : 2
701455 : 3
213236 : 2
1773 : 11018
1334075 : 21
1334048 : 20
1334068 : 21
1334077 : 21
1334072 : 21
1334071 : 20
1334070 : 21
1334073 : 21
1334057 : 21
1334052 : 21
1334061 : 21
1334058 : 21
1334051 : 20
1334053 : 21
1334065 : 21
1334047 : 21
1334083 : 21
1334082 : 21
1334078 : 20
1334055 : 20
78331 : 19
2583593 : 24
2583592 : 25
2583591 : 25
2583631 : 25
2583590 : 25
2583589 : 25
659019 : 25
1452723 : 25
1427516 : 25
1249615 : 21
443150 : 47
1010836 : 25
1010834 : 24
395095 : 24
1306414 : 21
443149 : 46
1310115 : 20
1304279 : 20
1138877 : 25
652616 : 24
83331 : 21
1765 : 636
2488963 : 2
2910957 : 2
2041533 : 1
2517948 : 1
2656573 : 10
2517942 : 24
2656603 : 2
2588495 : 10
2591077 : 9
2484212 : 12
20415

In [ ]:
input_fasta_folder_path_name = os.path.basename("home/hubert/TRS-omix_new/python/data/E_coli/E_coli")
base_results_directory = os.path.join(os.getcwd(), f"{input_fasta_folder_path_name}_results")
results_directory = base_results_directory
FileHandler.ensure_directory_exists(base_results_directory)
name_of_csv_file_storing_TRS_analysis_results = input_fasta_folder_path_name + "_results.csv"
path_of_folder_storing_TRS_analysis_results = os.path.join(base_results_directory, "TRS_output")
FileHandler.ensure_directory_exists(path_of_folder_storing_TRS_analysis_results)
path_of_csv_file_storing_TRS_analysis_results = os.path.join(path_of_folder_storing_TRS_analysis_results, 
                                                                 name_of_csv_file_storing_TRS_analysis_results)

In [ ]:
#Quick and dirty pair generation

In [2]:
input_fasta_folder_path = "/home/hubert/TRS-omix_new/python/data/E_coli/E_coli"
fasta_files = [f for f in os.listdir(input_fasta_folder_path) if f.endswith('.fasta') or f.endswith('.fa')]
trs_calculators = []
for fasta_file in fasta_files:
            path_to_input_fasta = os.path.join(input_fasta_folder_path, fasta_file)
            print(f"{path_to_input_fasta}")
            if not os.path.exists(path_to_input_fasta):
                print(f"File '{fasta_file}' does not exist! Skipping....")
                continue

            # Define the TRS file path
            trs_file = os.path.join(input_fasta_folder_path, 'trs.txt').encode('utf-8')
            # if not os.path.exists(trs_file):
            #     script_dir = os.path.dirname(os.path.abspath(__file__))
            #     trs_file = os.path.join(script_dir,'trs.txt')
            #     if not os.path.exists(trs_file):
            #         print("TRS file not found in both input folder and script location! Exiting...")
            #         break

            try:
                # Initialize TRS calculator for each sequence and perform TRS search
                trs_calculator = TRScalculator(sequence=path_to_input_fasta.encode('utf-8'), trs=trs_file, tmin=100, tmax=3000, mode=1)
                trs_calculator.calculate()
                trs_calculators.append(trs_calculator)
            except Exception as e:
                print(f"An error occurred while processing '{fasta_file}': {e}")
                continue

list_of_trs_results = []

for trs_calculator in trs_calculators:
    # Extract results from the calculator
    result = trs_calculator.Result
    # Append the result to the list
    list_of_trs_results.append(result)

/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/NC_018658.1.fasta
Encoded Sequence: b'/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/NC_018658.1.fasta'
Encoded TRS: b'/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/trs.txt'
Encoded Interiors: b'interiors.txt'
/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/NC_011741.1.fasta
Encoded Sequence: b'/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/NC_011741.1.fasta'
Encoded TRS: b'/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/trs.txt'
Encoded Interiors: b'interiors.txt'
/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/FM180568.1.fasta
Encoded Sequence: b'/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/FM180568.1.fasta'
Encoded TRS: b'/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/trs.txt'
Encoded Interiors: b'interiors.txt'
/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/NC_017626.1.fasta
Encoded Sequence: b'/home/hubert/TRS-omix_new/python/data/E_coli/E_coli/NC_017626.1.fasta'
Encoded TRS: b'/home/h

In [3]:
combined_trs_results = pd.concat(list_of_trs_results, ignore_index=True)
combined_trs_results[">SEQ"] = combined_trs_results[">SEQ"].str.replace(">","")
combined_trs_results
combined_trs_results
combined_trs_results = SequenceProcessor.extract_sequences(combined_trs_results, 50, 50)

Dataframe filtered successfully.


In [4]:
ncbi_ids = combined_trs_results["GENOME"].unique().tolist()
if Entrez.email and Entrez.email == 'hsalamaga@ibb.waw.pl' :
            print(f"Email adress is still set to {Entrez.email}")
organism_map = SequenceProcessor.fetch_organism_names(ncbi_ids,email = Entrez.email)

Fetching organism name for id: NC_018658.1


/home/hubert/miniconda3/envs/TRS/lib/python3.6/site-packages/Bio/Entrez/__init__.py:670: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  UserWarning,


Retrieved organism name for id: NC_018658.1 successfully Escherichia_coli_O104:H4_str._2011C-3493.
Fetching organism name for id: NC_011741.1
Retrieved organism name for id: NC_011741.1 successfully Escherichia_coli_IAI1.
Fetching organism name for id: FM180568.1
Retrieved organism name for id: FM180568.1 successfully Escherichia_coli_O127:H6_str._E2348/69.
Fetching organism name for id: NC_017626.1
Retrieved organism name for id: NC_017626.1 successfully Escherichia_coli_042.
Fetching organism name for id: NC_013361.1
Retrieved organism name for id: NC_013361.1 successfully Escherichia_coli_O26:H11_str._11368.
Fetching organism name for id: NC_011601.1
Retrieved organism name for id: NC_011601.1 successfully Escherichia_coli_O127:H6_str._E2348/69.
Fetching organism name for id: CP000801.1
Retrieved organism name for id: CP000801.1 successfully Escherichia_coli_O139:H28_str._E24377A.
Fetching organism name for id: CP007149.1
Retrieved organism name for id: CP007149.1 successfully Esche

In [5]:
combined_trs_results['Taxonomic Name'] = None
combined_trs_results['Taxonomic Name'] = combined_trs_results['GENOME'].map(organism_map)

In [6]:
combined_trs_results['L_id'] = combined_trs_results['Taxonomic Name'] + '_L' + combined_trs_results['L-No'].astype(str)
combined_trs_results['R_id'] = combined_trs_results['Taxonomic Name'] + '_R' + combined_trs_results['R-No'].astype(str)
sequences_df = combined_trs_results[['SEQ_L', 'SEQ_R', 'L_id', 'R_id']]

In [7]:
results_directory = 'home/hsalamaga/TRS-omix_new/python/test'
path_of_folder_storing_TRS_analysis_results = os.path.join(results_directory, "TRS_output")
os.makedirs(path_of_folder_storing_TRS_analysis_results, exist_ok=True) 
fasta_files_with_flanks = os.path.join(path_of_folder_storing_TRS_analysis_results, "combined_sequences.fasta")
with open(fasta_files_with_flanks, 'w') as fasta_file:
        for _, row in sequences_df.iterrows():
            # Write left sequence
            fasta_file.write(f'>{row["L_id"]}\n')
            fasta_file.write(f'{row["SEQ_L"]}\n')
            # Write right sequence
            fasta_file.write(f'>{row["R_id"]}\n')
            fasta_file.write(f'{row["SEQ_R"]}\n')

#Create unique FASTA file with renamed sequences (including ids and L/R identifiers)
fasta_files_with_flanks_unique = os.path.join(path_of_folder_storing_TRS_analysis_results, "combined_sequences_unique.fasta")
#SequenceProcessor.rename_sequences(fasta_files_with_flanks, fasta_files_with_flanks_unique)
SequenceProcessor.rename_sequences(fasta_files_with_flanks,fasta_files_with_flanks_unique)

Processing sequences: 100%|██████████| 58186/58186 [00:01<00:00, 31090.19it/s]


In [ ]:
#TRS proper
print("Starting analysis...")   
fasta_files = [f for f in os.listdir(input_fasta_folder_path_name) if f.endswith('.fasta') or f.endswith('.fa')]
trs_calculators = []

# Iterate over each FASTA file to calculate TRS sequences
for fasta_file in fasta_files:
    path_to_input_fasta = os.path.join(input_fasta_folder_path_name, fasta_file)
    print(f"{path_to_input_fasta}")
    if not os.path.exists(path_to_input_fasta):
        print(f"File '{fasta_file}' does not exist! Skipping....")
        continue

    # Define the TRS file path
    trs_file = os.path.join(args.input_fasta_folder_path, 'trs.txt').encode('utf-8')
    if not os.path.exists(trs_file):
        script_dir = os.path.dirname(os.path.abspath(__file__))
        trs_file = os.path.join(script_dir,'trs.txt')
        if not os.path.exists(trs_file):
            print("TRS file not found in both input folder and script location! Exiting...")
            break

    try:
        # Initialize TRS calculator for each sequence and perform TRS search
        trs_calculator = TRScalculator(sequence=path_to_input_fasta.encode('utf-8'), trs=trs_file, tmin=args.tmin, tmax=args.tmax, mode=args.mode)
        trs_calculator.calculate()
        trs_calculators.append(trs_calculator)
    except Exception as e:
        print(f"An error occurred while processing '{fasta_file}': {e}")
        continue

list_of_trs_results = []

In [ ]:
#fixing out of bounds errors 
results_directory = "/home/hubert/TRS-omix_new/python/E_coli_results_L100_R100_c1.0/"
combined_results = pd.read_csv("/home/hubert/TRS-omix_new/python/E_coli_results_L100_R100_c1.0/TRS_output/E_coli_results.csv")
filtered_fasta_file = FileHandler.find_file_by_name('unique_taxids_not_in_clusters_combined_sequences_unique_blastn_out.txt',folder= results_directory)
filtered_fasta_file = filtered_fasta_file[0]
#BLASTProcessor.extract_full_TRS_sequences(combined_results,filtered_fasta_file,results_directory,state=1)
print(combined_results.shape)
#might need to do the entire thing step by step

In [ ]:
#

In [ ]:
#GET SPECIES INFO
combined_results = pd.read_csv("/home/hubert/TRS-omix_new/python/klebsiella_results_L100_R100_c0.8/TRS_output/klebsiella_results.csv")

#Set the Entrez e-mail for further processing - seems like for some reason it refuses to remember it a this step and does not pass it into future ncbi requests
Entrez.email = SequenceProcessor.validate_and_set_email(email="hsalamaga@ibb.waw.pl")
print(f"Current email is set to {Entrez.email}")

#Fetch taxonomic information for the collected genome IDs
ncbi_ids = combined_results["GENOME"].unique().tolist()
tax_map = SequenceProcessor.fetch_organism_taxids(ncbi_ids)

#Filter and clean the taxonomic mapping
filtered_organism_taxid_map = {SequenceProcessor.filter_key_parts(key): value for key, value in tax_map.items()}
print(f"Species - taxid pairs detected in dataset : {filtered_organism_taxid_map}")

#Append taxid information to the filtered map
species_info = BLASTProcessor.append_taxids_to_filtered_map(filtered_organism_taxid_map)
species_info


#numbers assigned do not match the TRS_output file ????

In [ ]:
# Construct a dictionary of all taxid - acessions pairs in our data
modified_blast_path = "/home/hubert/TRS-omix_new/python/klebsiella_results_L100_R100_c0.8/blast_output/modified_blast/"
nan_file = "/home/hubert/TRS-omix_new/python/klebsiella_results_L100_R100_c0.8/blast_output/modified_blast/NaN acessions.csv"
results_dict = BLASTProcessor.construct_dict_from_files(modified_blast_path,nan_file)
results_dict

In [ ]:
combined_results = pd.read_csv("/home/hubert/TRS-omix_new/python/viruses_results_L100_R100_c0.8/TRS_output/viruses_results.csv")
filtered_fasta_file ="/home/hubert/TRS-omix_new/python/viruses_results_L100_R100_c0.8/blast_output_2nd_pass/modified_blast/unique_sequences/unique_taxids_full_sequences_blastn_out.txt"
results_directory = "/home/hubert/TRS-omix_new/python/viruses_results_L100_R100_c0.8/"
full_seq_final = "/home/hubert/TRS-omix_new/python/klebsiella_results_L100_R100_c0.8/final_results/full_sequences_final.fasta"
#BLASTProcessor.extract_full_TRS_sequences(combined_results,filtered_fasta_file,results_directory,state=2)


In [ ]:
#CREATE NEW CLUSTERS

cdhit_clusters_file = "/home/hubert/TRS-omix_new/python/viruses_results_L100_R100_c0.8/cd-hit-results/combined_sequences_unique_cdhit.clstr"
input_fasta = "/home/hubert/TRS-omix_new/python/viruses_results_L100_R100_c0.8/TRS_output/combined_sequences_unique.fasta"
output_folder_cluster = "/home/hubert/TRS-omix_new/python/viruses_results_L100_R100_c0.8/cd-hit-results/fasta_clusters"
results_directory = '/home/hubert/TRS-omix_new/python/klebsiella_results_L100_R100_c0.8/'
FileHandler.create_fasta_for_all_clusters(cdhit_clusters_file, input_fasta, output_folder_cluster, create_individual_files=True)
#test = create_trs_class_dataframe(output_folder_cluster)
#test

In [ ]:
#GENERTATE ADDITIONAL PLOTS
results_directory = "/home/hubert/TRS-omix_new/python/klebsiella_results_L100_R100_c0.8"
fasta_files_for_plotting = FileHandler.search_for_files(results_directory,'*.fasta')
fasta_files_for_plotting_names = FileHandler.extract_file_names(fasta_files_for_plotting)
print(f"Following fasta files were found in {results_directory} : {fasta_files_for_plotting_names}")
statistics = Stats()
for fasta_path in fasta_files_for_plotting:
    statistics.count_L_R(fasta_path)
file_paths = [file_path for file_path in fasta_files_for_plotting if file_path in statistics.file_info]
names_to_filter = ["full_sequences_final.fasta"]
file_paths = [file_path for file_path in file_paths if os.path.basename(file_path) in names_to_filter]
print(f"{file_paths}")
# Dictionary to store titles for each file path
file_titles = {}

# Populate file_titles dictionary using file_paths list
for file_path in file_paths:
# Extract file name from file path
    file_name = os.path.basename(file_path)

# Construct default title based on file name
default_title = f"Title for {file_name}"

# Add entry to file_titles dictionary
file_titles[file_path] = default_title

statistics.plot_lr_counts(file_paths=file_paths,file_titles=file_titles,results_directory=results_directory)

In [ ]:
#BINOMIAL TEST
output_folder_cluster = "/home/hubert/TRS-omix_new/python/Pseudomonas_aeruginosa100_results_L100_R100_c0.8/cd-hit-results/fasta_clusters" # valid name
full_seq_final = "/home/hubert/TRS-omix_new/python/Pseudomonas_aeruginosa100_results_L100_R100_c0.8/final_results/full_sequences_final.fasta" #unsure
df = Stats.create_trs_class_dataframe(output_folder_cluster) # lets work on naming the files as they occur at the end of the combined.py file
df = Stats.get_trs_class_totals(df) 
df

#This is used to extract the dataframe with total counts of classes from the full_seqeuences files
statistics = Stats()
df_full_fasta = statistics.create_lr_counts_dataframe(file_path=full_seq_final)
df_full_fasta

#Merge the dataframes
# def merge_trs_classes_results(df1,df2):
#     merged_df = pd.merge(df1,df2, left_index=True,right_index=True,how="outer")
#     total_sequences_found = merged_df['Total Count'].sum()/2
#     total_sequences_passed = merged_df['Counts'].sum()
#     merged_df["Proportion_Found"] = merged_df["Total Count"]/ total_sequences_found
#     merged_df["Proportion_Passed"] = merged_df["Counts"]/ total_sequences_passed
#     return merged_df

merged_df = Stats.merge_trs_classes_results(df_full_fasta,df)


final_results_folder = "/home/hubert/TRS-omix_new/python/Pseudomonas_aeruginosa100_results_L100_R100_c0.8/final_results/"
result_df = Stats.test_binomial_for_classes(merged_df)
result_df.index.name = "Class"
#Function for this now
# def save_binom_results(df,folder_path):
#     full_results = os.path.join(folder_path,"binomial_results_full.csv")
#     df.to_csv(full_results,index=True)
#     with_counts = df[(df["Counts"] >= 1) & (df["Expected Count"] <= df["Counts"])]
#     results_with_counts = os.path.join(folder_path,"binomial_results_with_counts.csv")
#     with_counts.to_csv(results_with_counts,index=True)
#     significant = df[df["P-Value"] <= 0.05]
#     results_significant = os.path.join(folder_path,"binomial_results_significant.csv")
#     significant.to_csv(results_significant,index=True)
#     return with_counts,significant

with_counts_df, significant_df = Stats.save_binom_results(result_df,final_results_folder)

#now let's think of a way to graph it 
#Probably pie chart ? 
# def aggregate_df(df,target,treshold):
#     df['Grouped'] = df[target].apply(lambda x: x if x / df[target].sum() > treshold else 'Other')
#     aggregated_df = df.groupby('Grouped')[target].sum()
#     return aggregated_df

# aggregated_df = aggregate_df(with_counts_df,"Counts",treshold=0.01)

# def plot_custom_pie(df, target):
#     explode = [0.1 if i == "L1" else 0 for i in df.index]
#     colors = ["#ff9999", "#66b3ff", "#99ff99", "#ffcc99"]


    
#     df[target].plot(
#         kind="pie",
#         figsize=(8, 8),
#         autopct=lambda pct: f'{pct:.1f}%\n({pct*df[target].sum()/100:.0f})',
#         startangle=90,
#         colors=colors,
#         explode=explode
#     )
#     plt.gca().set_aspect("equal")
#     plt.title("Customized TRS Class Distribution", fontsize=16)
#     plt.ylabel("")  # Remove default y-axis label
#     plt.tight_layout()
#     plt.show()



# #plot_custom_pie(with_counts_df,"Counts")
# aggregated_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_filtered_trs_heatmap(df, cluster_sizes, class_type='R'):
    """
    Plots a heatmap of TRS class counts for specific cluster sizes, separated by class type (R or L).

    Parameters:
        df (pd.DataFrame): DataFrame with TRS classes as rows and cluster sizes as columns.
        cluster_sizes (list): List of cluster sizes to include in the heatmap.
        class_type (str): 'R' for R classes, 'L' for L classes.
    """
    # Filter rows by class type
    filtered_rows = df[df.index.str.startswith(class_type)]

    # Filter columns by specified cluster sizes
    filtered_columns = [f"Cluster Size {size}" for size in cluster_sizes if f"Cluster Size {size}" in df.columns]
    if not filtered_columns:
        print("No matching cluster sizes found in DataFrame.")
        return

    filtered_df = filtered_rows[filtered_columns]

    # Plot the heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(filtered_df, annot=False, cmap="viridis", cbar_kws={'label': 'Frequency'})
    plt.title(f"TRS Class Distribution ({class_type} Classes) for Selected Cluster Sizes", fontsize=16)
    plt.xlabel("Cluster Size", fontsize=12)
    plt.ylabel("TRS Class", fontsize=12)
    plt.tight_layout()
    plt.show()

# Example Usage
plot_filtered_trs_heatmap(test, cluster_sizes=[1, 2, 5, 7], class_type='R')
#plot_filtered_trs_heatmap(test, cluster_sizes=[1, 2, 5], class_type='L')



In [ ]:
def plot_trs_bar(df, trs_classes=None, class_type='R', figsize=(16, 10), bar_width=0.8):
    """
    Plots a bar chart for selected TRS classes across cluster sizes, separated by class type (R or L),
    with adjustments for better spacing.

    Parameters:
        df (pd.DataFrame): DataFrame with TRS classes as rows and cluster sizes as columns.
        trs_classes (list): List of TRS classes to visualize. If None, shows all of the specified type.
        class_type (str): 'R' for R classes, 'L' for L classes.
        figsize (tuple): Size of the plot figure.
        bar_width (float): Width of the bars in the plot.
    """
    # Filter rows by class type
    filtered_df = df[df.index.str.startswith(class_type)]

    # Select specific TRS classes if provided
    if trs_classes:
        filtered_df = filtered_df.loc[trs_classes]

    # Transpose for better orientation
    filtered_df = filtered_df.transpose()

    # Plot the bar chart
    ax = filtered_df.plot(
        kind="bar",
        figsize=figsize,
        width=bar_width,
        legend=True
    )

    # Adjust labels and title
    plt.title(f"TRS Class Counts ({class_type} Classes)", fontsize=16)
    plt.xlabel("Cluster Size", fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.xticks(rotation=45, fontsize=10)
    plt.legend(title="TRS Class", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()


# Example Usage
plot_trs_bar(test, class_type='R')
plot_trs_bar(test, class_type='L')


In [ ]:
def plot_trs_stacked_bar(df, class_type='R'):
    """
    Plots a stacked bar chart of TRS class counts by cluster size, separated by class type (R or L).

    Parameters:
        df (pd.DataFrame): DataFrame with TRS classes as rows and cluster sizes as columns.
        class_type (str): 'R' for R classes, 'L' for L classes.
    """
    # Filter rows by class type
    filtered_df = df[df.index.str.startswith(class_type)]

    filtered_df.transpose().plot(kind="bar", stacked=True, figsize=(14, 8), cmap="tab20")
    plt.title(f"Stacked TRS Class Counts ({class_type} Classes)", fontsize=16)
    plt.xlabel("Cluster Size", fontsize=12)
    plt.ylabel("Total Count", fontsize=12)
    plt.xticks(rotation=45)
    plt.legend(title="TRS Class", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# Example Usage
plot_trs_stacked_bar(test, class_type='R')
plot_trs_stacked_bar(test, class_type='L')



In [ ]:
def plot_trs_line(df, trs_classes=None, class_type='R'):
    """
    Plots a line chart for selected TRS classes across cluster sizes, separated by class type (R or L).

    Parameters:
        df (pd.DataFrame): DataFrame with TRS classes as rows and cluster sizes as columns.
        trs_classes (list): List of TRS classes to visualize. If None, shows all of the specified type.
        class_type (str): 'R' for R classes, 'L' for L classes.
    """
    # Filter rows by class type
    filtered_df = df[df.index.str.startswith(class_type)]

    # Select specific TRS classes if provided
    if trs_classes:
        filtered_df = filtered_df.loc[trs_classes]

    filtered_df.transpose().plot(kind="line", figsize=(14, 8), marker='o')
    plt.title(f"TRS Class Trends ({class_type} Classes)", fontsize=16)
    plt.xlabel("Cluster Size", fontsize=12)
    plt.ylabel("Count", fontsize=12)
    plt.xticks(rotation=45)
    plt.legend(title="TRS Class", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

# Example Usage
plot_trs_line(test, class_type='R')
plot_trs_line(test, class_type='L')


In [ ]:
def plot_trs_pie(df, cluster_size):
    """
    Plots a pie chart of TRS class counts for a specific cluster size.

    Parameters:
        df (pd.DataFrame): DataFrame with TRS classes as rows and cluster sizes as columns.
        cluster_size (str): Cluster size column to visualize (e.g., "Cluster Size 1").
    """
    if cluster_size not in df.columns:
        print(f"Cluster size {cluster_size} not found in DataFrame.")
        return
    
    df[cluster_size].plot(kind="pie", figsize=(8, 8), autopct='%1.1f%%', startangle=140)
    plt.title(f"TRS Class Distribution for {cluster_size}", fontsize=16)
    plt.ylabel("")  # Remove y-axis label for pie chart
    plt.tight_layout()
    plt.show()

# Example Usage
plot_trs_pie(test, cluster_size="Cluster Size 4")


In [ ]:
'''Now let's try using cluster files to point out the clusters of given size in which all sequences belong to the same species 
then write out those clusters into separate .fasta files with names corresponding to species_cluster_size_ID,
 where ID is taken from initial file used for cluster filtering'''

In [ ]:
output_folder_cluster = "/home/hubert/TRS-omix_new/python/viruses_results_L100_R100_c0.8/cd-hit-results/fasta_clusters"


In [ ]:
import os

def find_single_species_clusters(folder_path, cluster_size=None):
    # If cluster_size is provided, restrict analysis to the specific folder
    if cluster_size:
        folder_path = os.path.join(folder_path, f"cluster_size_{cluster_size}")
        if not os.path.exists(folder_path):
            print(f"Folder for cluster size {cluster_size} does not exist.")
            return
    
    species_results = []
    
    # Define files to scan based on whether cluster_size is provided
    if cluster_size:
        # Analyze files directly in the specified folder
        files_to_scan = [
            (folder_path, file) for file in os.listdir(folder_path) if file.endswith(".fasta")
        ]
    else:
        # Analyze all files in subfolders
        files_to_scan = []
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)
            if not os.path.isdir(subfolder_path):  # Ignore files directly in folder_path
                continue
            files_to_scan.extend(
                [(subfolder_path, file) for file in os.listdir(subfolder_path) if file.endswith(".fasta")]
            )

    # Process each file
    for folder, file_name in files_to_scan:
        file_path = os.path.join(folder, file_name)
        with open(file_path, "r") as f:
            species_set = set()
            for line in f:
                if line.startswith(">"):
                    # Extract species name from header
                    species_parts = line.split()[0].split("_")[:-2]
                    species_name = "_".join(species_parts)
                    species_set.add(species_name)
            if len(species_set) == 1:
                species_name = species_set.pop()
                print(f"All sequences in {file_name} (in {folder}) are from the same species: {species_name}")
                species_results.append((folder, file_name, species_name))
    
    # Return results
    return species_results


In [ ]:
test = find_single_species_clusters(output_folder_cluster,cluster_size=2)

In [ ]:
test